In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from energyquantified import EnergyQuantified
from energyquantified.metadata import ContractPeriod
from statsmodels.tsa.seasonal import seasonal_decompose

from historicl_movement_functions.ttf_download_hisotry import ttf_download_hisotry_func
from datetime import datetime
from long_term_volatility_calc import *


In [11]:
api_key = 'ebbdc4-a81661-5dfb18-e24b2e'
http_proxy  = "http://proxy.spp.sk:8080"
https_proxy  = "http://proxy.spp.sk:8080"

proxies = {"http"  : http_proxy,
            "https"  : https_proxy}

eq = EnergyQuantified(api_key=api_key, proxies= proxies)
eq.is_api_key_valid()

True

In [12]:
import pulp
import random
# Sample forward price curve (e.g., 30 days of prices with a seasonal pattern)
import math
D = 30
prices = [0.5 + 0.4 * math.sin(2*math.pi * t / D) + 0.1 * (2*random.random() - 1) 
          for t in range(D)]  # synthetic prices with noise
prices = [round(p, 2) for p in prices]

# Storage parameters
min_inv = 0
max_inv = 500000
start_inv = 0
# Injection and withdrawal rate tiers:
# (We convert percentage fullness thresholds to absolute inventory amounts for constraints)
inj_thresholds = [0.30 * max_inv, 0.80 * max_inv]  # 150000, 400000
wd_threshold   = 0.25 * max_inv                    # 125000
# Max rates corresponding to tiers:
inj_rate = [6370, 5096, 3822]  # for <=30%, 30-80%, 80-100%
wd_rate  = [4430, 8860]        # for <=25%, >25%

# Initialize MILP model
model = pulp.LpProblem("GasStorageIntrinsic", pulp.LpMaximize)

# Decision variables
I = pulp.LpVariable.dicts("Inject", range(D), lowBound=0, upBound=inj_rate[0])      # injection volume per day
W = pulp.LpVariable.dicts("Withdraw", range(D), lowBound=0, upBound=wd_rate[1])    # withdrawal volume per day
Inv = pulp.LpVariable.dicts("Inv", range(D+1), lowBound=min_inv, upBound=max_inv)  # inventory at end of each day
y_inj1 = pulp.LpVariable.dicts("y_inj1", range(D), cat="Binary")
y_inj2 = pulp.LpVariable.dicts("y_inj2", range(D), cat="Binary")
y_inj3 = pulp.LpVariable.dicts("y_inj3", range(D), cat="Binary")
y_wd1  = pulp.LpVariable.dicts("y_wd1", range(D), cat="Binary")
y_wd2  = pulp.LpVariable.dicts("y_wd2", range(D), cat="Binary")

# Objective: maximize total profit = sum(W_d * (P_d - wd_cost) - I_d * (P_d + inj_cost))
inj_cost = 0.05
wd_cost = 0.03
model += pulp.lpSum([ W[d] * (prices[d] - wd_cost) - I[d] * (prices[d] + inj_cost) for d in range(D) ])

# Constraints:

# Initial inventory
model += Inv[0] == start_inv

# Inventory transition (balance) for each day
for d in range(D):
    model += Inv[d+1] == Inv[d] + I[d] - W[d], f"InventoryBalance_day{d}"

# Final inventory must be zero (end empty to realize all profits in horizon)
model += Inv[D] == 0

# Operational constraints per day
for d in range(D):
    # Injection rate limit based on chosen tier
    model += I[d] <= inj_rate[0] * y_inj1[d] + inj_rate[1] * y_inj2[d] + inj_rate[2] * y_inj3[d], f"InjRateLimit_day{d}"
    # Withdrawal rate limit based on chosen tier
    model += W[d] <= wd_rate[0] * y_wd1[d] + wd_rate[1] * y_wd2[d], f"WdRateLimit_day{d}"
    # Link inventory level to injection tier choice
    model += Inv[d] <= inj_thresholds[0] + max_inv * (1 - y_inj1[d]), f"InjTier1_if_day{d}"
    model += Inv[d] >= inj_thresholds[0] * y_inj2[d], f"InjTier2_min_day{d}"
    model += Inv[d] <= inj_thresholds[1] + max_inv * (1 - y_inj2[d]), f"InjTier2_max_day{d}"
    model += Inv[d] >= inj_thresholds[1] * y_inj3[d], f"InjTier3_if_day{d}"
    # Link inventory level to withdrawal tier choice
    model += Inv[d] <= wd_threshold + max_inv * (1 - y_wd1[d]), f"WdTier1_if_day{d}"
    model += Inv[d] >= wd_threshold * y_wd2[d], f"WdTier2_if_day{d}"
    # Only one injection tier and one withdrawal tier can be active (and at most one mode)
    model += y_inj1[d] + y_inj2[d] + y_inj3[d] <= 1, f"InjectTierChoice_day{d}"
    model += y_wd1[d] + y_wd2[d] <= 1, f"WithdrawTierChoice_day{d}"
    # Prevent simultaneous inject & withdraw
    model += y_inj1[d] + y_inj2[d] + y_inj3[d] + y_wd1[d] + y_wd2[d] <= 1, f"OneMode_day{d}"

# Solve the MILP
model.solve(pulp.PULP_CBC_CMD(msg=False))
print("Status:", pulp.LpStatus[model.status])
print("Total Profit =", pulp.value(model.objective))


Status: Optimal
Total Profit = 6668.2
